In [169]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import datetime 
from scipy import stats
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import csv
import time
import math
from collections import Counter
from collections import Counter 

#most_frequent_error在取相同數量的眾數時有錯，不應該取第一個
def most_frequent_error(List): 
    occurence_count = Counter(List) 
    return occurence_count.most_common(1)[0][0] 

def most_frequent(List): 
    occurence_count = Counter(List) 
    if (len(Counter(List))) == 1:
        return occurence_count.most_common(1)[0][0]
    else:
        if occurence_count.most_common(1)[0][1] == occurence_count.most_common(2)[1][1]:
            return 'no_mode'
        else:
            return occurence_count.most_common(1)[0][0]


## 選擇時間範圍

In [170]:
file_name = 'SP1081_10_17_final_chap'

data = pd.read_csv(r'C:/Users/KSLab/Google 雲端硬碟/中原/sequence/' + file_name + '.csv')
data["begin_time"] = pd.to_datetime(data["begin_time"], format='%Y-%m-%d %H:%M:%S')
data["over_time"] = pd.to_datetime(data["over_time"], format='%Y-%m-%d %H:%M:%S')

# 退選、助教、教師名單
data =  data[(data['username'] != 10138) & (data['username'] != '10138') &
             (data['username'] != 'sunnet')
            &  (data['username'] != 71977) &  (data['username'] != '71977') 
             & (data['username'] != 10777026)  & (data['username'] != '10777026')]


data = data.sort_values(['username','begin_time'])
data.reset_index(inplace=True, drop = True)  #記得reset index
data.head(10)

,username,begin_time,course_id,over_time,title,章節
0,10327225,2019-11-15 17:51:00,10108490.0,2019-11-15 17:51:00,完整教材錄影,b
1,10327225,2019-11-15 17:51:00,10108490.0,2019-11-15 17:55:00,完整投影片,NaN
2,10327225,2019-11-15 17:55:00,10108490.0,2019-11-15 22:21:00,完整投影片,NaN
3,10327225,2019-11-29 11:27:00,10108490.0,2019-11-29 11:27:00,完整投影片,NaN
4,10327225,2019-11-29 11:27:00,10108490.0,2019-11-29 11:28:00,完整投影片,NaN
5,10327225,2019-11-29 11:28:00,10108490.0,2019-11-29 11:29:00,完整投影片,NaN
6,10327225,2019-11-29 11:30:00,10108490.0,2019-11-29 11:30:00,完整教材錄影,a
7,10327225,2019-11-29 11:31:00,10108490.0,2019-11-29 11:33:00,完整教材錄影,a
8,10327225,2019-11-29 11:32:00,10108490.0,2019-11-29 11:33:00,完整教材錄影,NaN
9,10327225,2019-11-29 11:33:00,10108490.0,2019-11-29 11:34:00,討論,NaN


# 序列化

In [171]:
seq_list = []
chap_list = []
seq_num = -1


for name in data['username'].unique():
    #序列初始化
    seq_list.append([])
    chap_list.append([])
    seq_num+=1
    
    #分成每一個人
    person_data = data[data['username'] == name]
    person_data.reset_index(inplace=True, drop = True) 
    opentime = person_data['begin_time'][0]
    seq_list[seq_num].append(person_data['username'][0]) #加入姓名
    for i in range(len(person_data)):
        #超過3小時斷開
        #這裡有一個坑，如果不同天的相減.seconds不會計算到跨天數
        if ((person_data['begin_time'][i] - opentime).seconds >= 10800) or ((person_data['begin_time'][i] - opentime).days  >= 1 ) :  
            opentime = person_data['begin_time'][i]
            seq_list.append([])
            chap_list.append([])
            seq_num+=1
            seq_list[seq_num].append(person_data['username'][i]) #加入姓名
            seq_list[seq_num].append(person_data['title'][i])
            chap_list[seq_num].append(person_data['章節'][i])
        else:
            #opentime = person_data['begin_time'][0] 多這一行是動作與動作間相隔不超過3小時
            seq_list[seq_num].append(person_data['title'][i])
            chap_list[seq_num].append(person_data['章節'][i])
            
seq_list      

[['10327225', '完整教材錄影', '完整投影片', '完整投影片'],
 ['10327225',
  '完整投影片',
  '完整投影片',
  '完整投影片',
  '完整教材錄影',
  '完整教材錄影',
  '完整教材錄影',
  '討論',
  '教材下載',
  '教材下載',
  '教材下載',
  '教材下載',
  '教材下載',
  '教材下載',
  '教材下載',
  '教材下載',
  '教材下載',
  '教材下載',
  '教材下載'],
 ['10327225', '相關規定'],
 ['10327225', '教材下載'],
 ['10427106', '完整投影片'],
 ['10427106', '完整投影片', '完整投影片', '完整投影片'],
 ['10427106', '完整投影片'],
 ['10427106', '完整投影片', '測驗'],
 ['10427106', '討論'],
 ['10427106', '完整教材錄影', '完整教材錄影', '完整教材錄影', '完整教材錄影', '完整教材錄影'],
 ['10427106', '完整教材錄影', '完整教材錄影'],
 ['10427106', '完整教材錄影'],
 ['10427106', '完整教材錄影'],
 ['10427106',
  '完整教材錄影',
  '教材下載',
  '討論',
  '討論',
  '討論',
  '討論',
  '討論',
  '討論',
  '討論',
  '討論'],
 ['10427125', '完整教材錄影', '完整投影片', '完整教材錄影'],
 ['10427125', '完整教材錄影', '重點整理'],
 ['10427125', '重點整理', '完整教材錄影', '完整教材錄影'],
 ['10427125', '完整教材錄影'],
 ['10427125', '完整教材錄影'],
 ['10427125', '完整教材錄影'],
 ['10427125', '完整教材錄影'],
 ['10427125', '完整教材錄影'],
 ['10427125', '完整教材錄影', '完整教材錄影'],
 ['10427125', '完整教材錄影'],
 ['10427125'

In [172]:
#計算序列中最多章節並加入資訊

for i in range(len(chap_list)):
    while np.nan in chap_list[i]:
        chap_list[i].remove(np.nan) 
    if len(chap_list[i]) == 0:
        seq_list[i].append('Nan')
    else:
        seq_list[i].append(str(most_frequent(chap_list[i]))) #與舊的程式碼最主要差在這裡，mode方程式的選擇

In [173]:
# 分群結果合併 去章節標示
merge_list = []
seq= 0
for row in range(len(seq_list)):
    if row == 0:
        merge_list.append([])
        for i in range(0,len(seq_list[row])-1):
            merge_list[seq].append(seq_list[row][i])        
    
    elif row != 0 and  seq_list[row][0] == seq_list[row-1][0] and seq_list[row][len(seq_list[row])-1] == seq_list[row-1][len(seq_list[row-1])-1]:
        for i in range(1,len(seq_list[row])-1):
            merge_list[seq].append(seq_list[row][i])
            
    else:
        merge_list.append([])
        seq = seq+1
        for i in range(0,len(seq_list[row])-1):
            merge_list[seq].append(seq_list[row][i])  

In [174]:
# Recode
for row in range(len(merge_list)):
    check = 0
    while (not check):
        for col in range(len(merge_list[row])):
            if(col!= len(merge_list[row])-1):
                if merge_list[row][col] == merge_list[row][col+1] :
                    del merge_list[row][col+1]
                    break
            elif(col == len(merge_list[row])-1):
                check = 1

# 去除離群值 Q1-1.5*IQR Q3+1.5*IQR

In [175]:
import math

count = 0
nums = []
max = 0

for i in range(len(merge_list)):
    
    count = len(merge_list[i])-1 + count
    nums.append(len(merge_list[i])-1)
    if len(merge_list[i])-1 > max:
        max = len(merge_list[i])-1
        
IQR = np.percentile(nums, [75]) - np.percentile(nums, [25])
        
print('*'*20, 'Raw sequence ','*'*20)     
print('平均長度 = ', count/len(merge_list)) 


print ('四分位數Q1 Q2 Q3', np.percentile(nums, [25, 50, 75]) )
print ('四分位距IQR = ',  IQR)

Q3 = np.percentile(nums, [75])


Q1_IQR = np.percentile(nums, [25]) - 1.5*IQR
Q1_IQR = math.floor(Q1_IQR) 

Q3_IQR = np.percentile(nums, [75]) + 1.5*IQR
Q3_IQR = math.floor(Q3_IQR) 


print ('Q1-1.5*IQR = ',  Q1_IQR) 
print ('Q3+1.5*IQR = ',  Q3_IQR)

print('序列共有 = ', len(merge_list)) 
print('最長 = ', max) 

******************** Raw sequence  ********************
平均長度 =  2.931297709923664
四分位數Q1 Q2 Q3 [1. 2. 4.]
四分位距IQR =  [3.]
Q1-1.5*IQR =  -4
Q3+1.5*IQR =  8
序列共有 =  655
最長 =  63


In [176]:
#找outlier

Q1_outlier = 0
Q3_outlier = 0

for row in range(len(merge_list)):
    if len(merge_list[row]) >= Q3_IQR + 1 :
        Q3_outlier+=1
        merge_list[row] = 'null'
        
    if len(merge_list[row]) <= Q1_IQR + 1 :
        Q1_outlier+=1
        merge_list[row] = 'null'
        
print('Q1-1.5*IQR',Q1_outlier,'Q3+1.5*IQR',Q3_outlier)

while 'null' in merge_list:
    merge_list.remove('null') 


Q1-1.5*IQR 0 Q3+1.5*IQR 34


In [177]:
merge_data = pd.DataFrame(data=merge_list)
for i in range(len(merge_data)):
    if merge_data.iloc[i,1] == None:
        merge_data = merge_data.drop([i])     
        
merge_data = merge_data.fillna('NA')
merge_data.to_csv(r'C:/Users/KSLab/Google 雲端硬碟/中原/sequence/'+ file_name +'_result_.csv', index = False)

merge_data = merge_data.drop(merge_data.columns[0], axis=1)
merge_data.to_csv(r'C:/Users/KSLab/Google 雲端硬碟/中原/sequence/'+ file_name + '_result_nomame.csv', index = False)

#################算長度,中位數Q1 Q2 Q3#######


count = 0
nums = []
max = 0

for i in range(len(merge_list)):
    
    count = len(merge_list[i])-1 + count
    nums.append(len(merge_list[i])-1)
    if len(merge_list[i])-1 > max:
        max = len(merge_list[i])-1
    
print('*'*20, 'preprocessing sequence ','*'*20)     
print('平均長度 = ', count/len(merge_list)) 
print ('四分位數Q1 Q2 Q3', np.percentile(nums, [25, 50, 75]) )
print('序列共有 = ', len(merge_list)) 
print('最長 = ', max) 

******************** preprocessing sequence  ********************
平均長度 =  2.360708534621578
四分位數Q1 Q2 Q3 [1. 2. 3.]
序列共有 =  621
最長 =  7
